---
layout: post
title: "The Case Against Ranked-Choice Voting: Part I"
tags: [Elections, Politics]
--- 

There has been increased interest recently in exploring voting systems beyond the first-past-the-post (FPTP) system so common in US elections. Perhaps the most popular alternative is ranked-choice voting. Although I prefer ranked-choice voting to FPTP, I think there are many problems with this system that warrant looking for a different system. In this post, I argue the case against ranked-choice voting, pointing out that it has many properties that are undesirable for a voting system.

This post discusses these problems in theoretical elections. In the following post, I talked about how these can or have happened in real elections. In a third post, I will discuss approval voting and how that remedies many of the issues with ranked-choice voting.

<b>Table of contents</b>
* TOC
{:toc}

Ranked-choice voting, also known as instant runoff voting, is a voting system where voters rank their choice of candidates. The vote counting consists of multiple rounds where the vote totals are counted. If no candidate wins the majority of the votes after a round, candidates with the least votes are eliminated and the votes then go to the next candidate on the ranked ballot. Thus it is like having an instant runoff election, which is why this process is also known as instant runoff voting. There are lots of variations to this general approach (like how many candidates to remove each round), but this is the general idea.

Proponents of ranked-choice voting, such as [FairVote](https://www.fairvote.org/) make many claims about the benefits of ranked-choice voting. They claim that it reduces strategic voting, provides more choice because voters don't have to worry about vote splitting, and prevents 3rd-party candidates from "spoiling" an election.

Unfortunately, I don't think all these claims are true. Many of these negative effects still happen with ranked-choice voting. I want to be clear that I think ranked-choice voting is far better than first-past-the-post voting, but it still can be improved on. In particular, my preferred voting style is approval voting, which I will talk about in another post.

In this post, I will be simulating elections using [VoteSim](https://github.com/jss367/votesim), which is a free and open-source library for simulating elections. The code to run the simulations in this blog post is also available.

# Drawbacks of Ranked-Choice Voting Systems

## Ranked-Choice Voting Encourages Strategic Voting

Proponents claim that ranked-choice voting encourages voters to vote their preference, but in this simulation I'll show that it encourages strategic voting.

Let's start with a simple election between three candidates: Alice, Bob, and Charlie. Let's not worry about specific political parties, we'll just say that Alice and Charlie are in opposing camps and Bob is somewhere in the middle. The election is for a single seat.

Let's imagine there are ten voters. Four of the voters prefer Alice, four prefer Charlie, and two prefer Bob. All voters who prefer Alice or Charlie prefer the Bob candidate to the other camp's candidate. Those that prefer Bob prefer Alice as the second-choice candidate.

The voters have the following preferences:

* Alice, Bob, Charlie
* Alice, Bob, Charlie
* Alice, Bob, Charlie
* Alice, Bob, Charlie
* Charlie, Bob, Alice
* Charlie, Bob, Alice
* Charlie, Bob, Alice
* Charlie, Bob, Alice
* Bob, Charlie, Alice
* Bob, Charlie, Alice

Here are the ballots, where the first name is the first preference of the voter:

* Ranked Ballot: Alice, Bob
* Ranked Ballot: Alice, Bob
* Ranked Ballot: Alice, Bob
* Ranked Ballot: Alice, Bob
* Ranked Ballot: Bob, Alice
* Ranked Ballot: Bob, Alice
* Ranked Ballot: Charlie, Bob
* Ranked Ballot: Charlie, Bob
* Ranked Ballot: Charlie, Bob
* Ranked Ballot: Charlie, Bob

Let's look at how this election would play out using ranked-choice voting.

In [1]:
from votesim import *

In [2]:
Alice = Candidate("Alice")
Bob = Candidate("Bob")
Charlie = Candidate("Charlie")

candidates = [Alice, Bob, Charlie]

ballots = [
    Ballot(ranked_candidates=[Alice, Bob]),
    Ballot(ranked_candidates=[Alice, Bob]),
    Ballot(ranked_candidates=[Alice, Bob]),
    Ballot(ranked_candidates=[Alice, Bob]),
    Ballot(ranked_candidates=[Bob, Alice]),
    Ballot(ranked_candidates=[Bob, Alice]),
    Ballot(ranked_candidates=[Charlie, Bob]),
    Ballot(ranked_candidates=[Charlie, Bob]),
    Ballot(ranked_candidates=[Charlie, Bob]),
    Ballot(ranked_candidates=[Charlie, Bob]),
]

election_result = instant_runoff_voting(candidates, ballots)
election_result.get_winners()
print(election_result)

ROUND 1
Candidate      Votes  Status
-----------  -------  --------
Alice              4  Active
Charlie            4  Active
Bob                2  Rejected

FINAL RESULT
Candidate      Votes  Status
-----------  -------  --------
Alice              6  Elected
Charlie            4  Rejected
Bob                0  Rejected



Alice won and the Charlie voters are furious. They would have been fine with Bob, but are not happy with Alice. The next election comes up with the same voters and the same candidates, but now, Charlie's voters are going to be more strategic.

They know what happened last time and they want to stop it from happening again. So two of the Charlie supporter change their ballots to vote for Bob as their first candidate and Charlie as their second candidate.

* Ranked Ballot: Alice, Bob
* Ranked Ballot: Alice, Bob
* Ranked Ballot: Alice, Bob
* Ranked Ballot: Alice, Bob
* Ranked Ballot: Bob, Alice
* Ranked Ballot: Bob, Alice
* Ranked Ballot: Bob, Charlie
* Ranked Ballot: Bob, Charlie
* Ranked Ballot: Charlie, Bob
* Ranked Ballot: Charlie, Bob

In [3]:
Alice = Candidate("Alice")
Bob = Candidate("Bob")
Charlie = Candidate("Charlie")

candidates = [Alice, Bob, Charlie]

ballots = [
    Ballot(ranked_candidates=[Alice, Bob]),
    Ballot(ranked_candidates=[Alice, Bob]),
    Ballot(ranked_candidates=[Alice, Bob]),
    Ballot(ranked_candidates=[Alice, Bob]),
    Ballot(ranked_candidates=[Bob, Alice]),
    Ballot(ranked_candidates=[Bob, Alice]),
    Ballot(ranked_candidates=[Bob, Charlie]),
    Ballot(ranked_candidates=[Bob, Charlie]),
    Ballot(ranked_candidates=[Charlie, Bob]),
    Ballot(ranked_candidates=[Charlie, Bob]),
]

election_result = instant_runoff_voting(candidates, ballots)
election_result.get_winners()
print(election_result)

ROUND 1
Candidate      Votes  Status
-----------  -------  --------
Bob                4  Active
Alice              4  Active
Charlie            2  Rejected

FINAL RESULT
Candidate      Votes  Status
-----------  -------  --------
Bob                6  Elected
Alice              4  Rejected
Charlie            0  Rejected



Now Bob wins the election. This is what the Charlie voters wanted, but they got it by voting *against* their preferences. Thus, in some cases, ranked-choice voting *encourages* people to vote differently than their preferences.

## Ranked-Choice Voting Doesn't Always Result in the Best Head-to-head Candidate

There's another problem with the initial election. Let's go back to the ballots that got Alice elected.

In [4]:
ballots = [
    Ballot(ranked_candidates=[Alice, Bob]),
    Ballot(ranked_candidates=[Alice, Bob]),
    Ballot(ranked_candidates=[Alice, Bob]),
    Ballot(ranked_candidates=[Alice, Bob]),
    Ballot(ranked_candidates=[Bob, Alice]),
    Ballot(ranked_candidates=[Bob, Alice]),
    Ballot(ranked_candidates=[Charlie, Bob]),
    Ballot(ranked_candidates=[Charlie, Bob]),
    Ballot(ranked_candidates=[Charlie, Bob]),
    Ballot(ranked_candidates=[Charlie, Bob]),
]

One would hope that the election would result in the candidate who would wins beating the other candidates if there were a head-to-head election. But under ranked-choice voting, this is not the case. In the election above, Alice won, but in a head-to-head election, Bob would have won against either Alice or Charlie.

In [5]:
prefer_alice = 0
prefer_bob = 0
for ballot in ballots:
    for candidate in ballot.ranked_candidates:
        if candidate == Alice:
            prefer_alice += 1
            break
        elif candidate == Bob:
            prefer_bob += 1
            break

In [6]:
print(f"In a head-to-head election, {prefer_bob} voters prefer Bob and {prefer_alice} voters prefer Alice")

In a head-to-head election, 6 voters prefer Bob and 4 voters prefer Alice


In [7]:
prefer_bob = 0
prefer_charlie = 0
for ballot in ballots:
    for candidate in ballot.ranked_candidates:
        if candidate == Bob:
            prefer_bob += 1
            break
        elif candidate == Charlie:
            prefer_charlie += 1
            break

In [8]:
print(f"In a head-to-head election, {prefer_bob} voters prefer Bob and {prefer_charlie} voters prefer Charlie")

In a head-to-head election, 6 voters prefer Bob and 4 voters prefer Charlie


## Ranked-Choice Voting Allows for Election Spoilers

Being that the ranked-choice voting results aren't the same as in head-to-head matchups, this also means that spoiler candidates will exist. To show that, let's take the previous election but instead, we'll say Charlie didn't run. How would that affect the results?

In [9]:
Alice = Candidate("Alice")
Bob = Candidate("Bob")

candidates = [Alice, Bob]

ballots = [
    Ballot(ranked_candidates=[Alice, Bob]),
    Ballot(ranked_candidates=[Alice, Bob]),
    Ballot(ranked_candidates=[Alice, Bob]),
    Ballot(ranked_candidates=[Alice, Bob]),
    Ballot(ranked_candidates=[Bob, Alice]),
    Ballot(ranked_candidates=[Bob, Alice]),
    Ballot(ranked_candidates=[Bob]),
    Ballot(ranked_candidates=[Bob]),
    Ballot(ranked_candidates=[Bob]),
    Ballot(ranked_candidates=[Bob]),
    Ballot(ranked_candidates=[Bob]),
]

election_result = instant_runoff_voting(candidates, ballots)
election_result.get_winners()
print(election_result)

FINAL RESULT
Candidate      Votes  Status
-----------  -------  --------
Bob                7  Elected
Alice              4  Rejected



Bob wins. But when Charlie was in the election, Alice won. Therefore the existance of Charlie spoiled the election for Bob.

## Ranked-Choice Voting Allows for Candidates to Do Better but Receive Worse Outcomes

This is a very odd situation and it took me looking at the data for a bit before I believed it. But there are times when a candidate can perform better and end up doing worse in an election. Here's a scenario:

Let's look at the case of a primary election with four districts in it, each with a candidate hailing from that district. The four candidates are Alice, Bob, Charlie, and Dan. Alice is from district 1, the largest district, has lots of experience, and is considered the frontrunner for the race. Bob is from district 2, which is slightly smaller, but also has lots of experience and is the other main contender. Charlie is from district 3, which is smaller than 1 or 2. He isn't well-known outside of his district. Dan is the odd-ball candidate from district 4, which is tiny compared to the others. He is popular in district 4 but completely unknown outside of it.

District 1 voters prefer the mainstream candidates, Alice and Bob. They have seven voters who all rank the candidates: Alice, Bob, Charlie, Dan.

District 2 voters are similar to district 1 voters, except that they prefer their hometown candidate Bob. They have six voters who all rank the candidates: Bob, Alice, Charlie, Dan

District 3 is smaller than the other districts but still well-represented. The voters their prefer their candidate, Charlie, but would also accept Bob or Alice. Out of those two, they prefer Bob because they see district 1 as too large and powerful already and don't want district 1 to have too much power. They have five voters who use the following ranking: Charlie, Bob, Alice, Dan

District 4 is much smaller than the others and the voters there feel completely ignored by the mainstream candidates. The elections have gone to candidates from districts 1 and 2 for ages and they want something different. Their ranking is: Dan, Charlie, Bob, Alice

In [10]:
alice = Candidate("Alice")
Bob = Candidate("Bob")
Charlie = Candidate("Charlie")
Dan = Candidate("Dan")

candidates = [Alice, Bob, Charlie, Dan]

ballots = [
    Ballot(ranked_candidates=[Alice, Bob, Charlie, Dan]),
    Ballot(ranked_candidates=[Alice, Bob, Charlie, Dan]),
    Ballot(ranked_candidates=[Alice, Bob, Charlie, Dan]),
    Ballot(ranked_candidates=[Alice, Bob, Charlie, Dan]),
    Ballot(ranked_candidates=[Alice, Bob, Charlie, Dan]),
    Ballot(ranked_candidates=[Alice, Bob, Charlie, Dan]),
    Ballot(ranked_candidates=[Alice, Bob, Charlie, Dan]),
    Ballot(ranked_candidates=[Bob, Alice, Charlie, Dan]),
    Ballot(ranked_candidates=[Bob, Alice, Charlie, Dan]),
    Ballot(ranked_candidates=[Bob, Alice, Charlie, Dan]),
    Ballot(ranked_candidates=[Bob, Alice, Charlie, Dan]),
    Ballot(ranked_candidates=[Bob, Alice, Charlie, Dan]),
    Ballot(ranked_candidates=[Bob, Alice, Charlie, Dan]),
    Ballot(ranked_candidates=[Charlie, Bob, Alice, Dan]),
    Ballot(ranked_candidates=[Charlie, Bob, Alice, Dan]),
    Ballot(ranked_candidates=[Charlie, Bob, Alice, Dan]),
    Ballot(ranked_candidates=[Charlie, Bob, Alice, Dan]),
    Ballot(ranked_candidates=[Charlie, Bob, Alice, Dan]),
    Ballot(ranked_candidates=[Dan, Charlie, Bob, Alice]),
    Ballot(ranked_candidates=[Dan, Charlie, Bob, Alice]),
    Ballot(ranked_candidates=[Dan, Charlie, Bob, Alice]),
]

election_result = instant_runoff_voting(candidates, ballots)
election_result.get_winners()
print(election_result)


ROUND 1
Candidate      Votes  Status
-----------  -------  --------
Alice              7  Active
Bob                6  Active
Charlie            5  Active
Dan                3  Rejected

ROUND 2
Candidate      Votes  Status
-----------  -------  --------
Charlie            8  Active
Alice              7  Active
Bob                6  Rejected
Dan                0  Rejected

FINAL RESULT
Candidate      Votes  Status
-----------  -------  --------
Alice             13  Elected
Charlie            8  Rejected
Bob                0  Rejected
Dan                0  Rejected



Alice wins the election.

The next election comes along and it's the same candidates and the same voters. However, this campaign Alice decided to reach out to District 4 and put extra effort into getting their votes. Through her hard work, she convinces them to put her second (behind Dan) on the ballot. So the voters from District 4 preferences are now: Dan, Alice, Charlie, Bob. Given that "Alice" won the previous election and this change should only help, how would this affect the results?

In [11]:

Alice = Candidate("Alice")
Bob = Candidate("Bob")
Charlie = Candidate("Charlie")
Dan = Candidate("Dan")

candidates = [Alice, Bob, Charlie, Dan]

ballots = [
    Ballot(ranked_candidates=[Alice, Bob, Charlie, Dan]),
    Ballot(ranked_candidates=[Alice, Bob, Charlie, Dan]),
    Ballot(ranked_candidates=[Alice, Bob, Charlie, Dan]),
    Ballot(ranked_candidates=[Alice, Bob, Charlie, Dan]),
    Ballot(ranked_candidates=[Alice, Bob, Charlie, Dan]),
    Ballot(ranked_candidates=[Alice, Bob, Charlie, Dan]),
    Ballot(ranked_candidates=[Alice, Bob, Charlie, Dan]),
    Ballot(ranked_candidates=[Bob, Alice, Charlie, Dan]),
    Ballot(ranked_candidates=[Bob, Alice, Charlie, Dan]),
    Ballot(ranked_candidates=[Bob, Alice, Charlie, Dan]),
    Ballot(ranked_candidates=[Bob, Alice, Charlie, Dan]),
    Ballot(ranked_candidates=[Bob, Alice, Charlie, Dan]),
    Ballot(ranked_candidates=[Bob, Alice, Charlie, Dan]),
    Ballot(ranked_candidates=[Charlie, Bob, Alice, Dan]),
    Ballot(ranked_candidates=[Charlie, Bob, Alice, Dan]),
    Ballot(ranked_candidates=[Charlie, Bob, Alice, Dan]),
    Ballot(ranked_candidates=[Charlie, Bob, Alice, Dan]),
    Ballot(ranked_candidates=[Charlie, Bob, Alice, Dan]),
    Ballot(ranked_candidates=[Dan, Alice, Charlie, Bob]),
    Ballot(ranked_candidates=[Dan, Alice, Charlie, Bob]),
    Ballot(ranked_candidates=[Dan, Alice, Charlie, Bob]),
]

election_result = instant_runoff_voting(candidates, ballots)

winners = election_result.get_winners()

print(election_result)


ROUND 1
Candidate      Votes  Status
-----------  -------  --------
Alice              7  Active
Bob                6  Active
Charlie            5  Active
Dan                3  Rejected

ROUND 2
Candidate      Votes  Status
-----------  -------  --------
Alice             10  Active
Bob                6  Active
Charlie            5  Rejected
Dan                0  Rejected

FINAL RESULT
Candidate      Votes  Status
-----------  -------  --------
Bob               11  Elected
Alice             10  Rejected
Charlie            0  Rejected
Dan                0  Rejected



Stunningly, this change has knocked Alice out of office. The only change here is that voters voted Alice *higher*, but now she's lost an election that she otherwise would have won.

## Results from Real Races

These results are more than merely theoretical. My next post will contain election results from the 2009 election for mayor of Burlington, Vermont.